# Practice Test

## 1.  Train KNN on breast cancer data set
1.  Here's a link to the breast cancer data set on UCI data repository.  [breast cancer data](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Prognostic%29)
1.  Data inspection check data for missing values.  
-Make a decision about whether to discard rows with missing data or to try and fill in the missing values.  
Hint:  If you decide to fill in the missing values, the easiest method is to replace missing values is to substitute the average of the non-missing values
2.  Set up data for training, validation and testing.  
-You'll use n-fold cross validation to determine k in k nearest neighbors.  Hold out 20% of the data for final testing.  

3.  Scale data
4.  Produce plots of absolute value of error for each of the n validation folds as a function of complexity (number of training examples minus k).  Also plot the root mean squared error.  Put all of these plots on the same graph.  
5.  Choose the best value of k for KNN and generate error on the test set to confirm your validation results. 

Here are some examples of what a training and validation error curves look like: [some train/validation plot examples](https://www.google.com/search?q=train+and+test+error+plot+images&biw=1580&bih=864&tbm=isch&imgil=Y7yivsIwgm2AXM%253A%253B8QRxV4RA1cfQ1M%253Bhttp%25253A%25252F%25252Fgenomicsclass.github.io%25252Fbook%25252Fpages%25252Fmachine_learning.html&source=iu&pf=m&fir=Y7yivsIwgm2AXM%253A%252C8QRxV4RA1cfQ1M%252C_&usg=__IkOEpQtI4rjyBaGTQqOOhAKvA4E%3D&ved=0ahUKEwiI9cS4o5fQAhWo0FQKHcT1DxgQyjcINA&ei=N8ggWMj9H6ih0wLE67_AAQ#imgrc=Pz8e50_l-ZGWrM%3A)

Note:  With KNN and with elastic net (described below), you'll treat the breast cancer problem as a regression problem.  That is train your model to predict the real number labels (2 and 4).  Measure and report performance in real number terms (sample standard deviation of error, for example).  We'll talk about approaches to dealing with classification error in the next lecture. 

Pseudo code looks something like:

In [19]:
# import pandas as pd
import numpy as np

with open('breast-cancer-wisconsin.data.txt') as f:
    data = f.readlines()
    
# df = pd.read_csv('breast-cancer-wisconsin.data.txt')
# df.shape()
# df.dropna()

cleaned_list= []

for x in data:
    if "?" not in x:
        cleaned_list.append(x)
#     else:
#         print(x)
        
# cleaned_list.split(",")

# for x in cleaned_list:
#     x.split(",")

# for i in range(len(cleaned_list))
#     cleaned_list[i].replace("\n", "").split(",")

list_of_lists = []

for x in cleaned_list:
    temp = x.replace("\n", "").split(",")
    row = [float(z) for z in temp]
    list_of_lists.append(row)
    
list_of_lists = np.array(list_of_lists)
list_of_lists.shape
list_of_lists[:,1:].shape

no_ids = list_of_lists[:,1:]
X = no_ids[:,:-1]
y = no_ids[:,-1]  # labels
# x.shape
# y.shape

In [20]:
from sklearn.model_selection import train_test_split
xTrVal, xTe, yTrVal, yTe = train_test_split(X, y, test_size =.2)  #Function for splitting off data for final test
nFolds = 5  #number of cross validation folds
xTrVal.shape, xTe.shape, yTrVal.shape, yTe.shape

((546, 9), (137, 9), (546,), (137,))

In [23]:
# cross validation loop  
for ixVal in range(nFolds):
    valIdx = [i for i in range(len(xTrVal)) if i%nFolds==ixVal]  #or use sklearn k-fold package
    trainIdx = [i for i in range(len(xTrVal)) if i%nFolds!=ixVal]
    xTrain = xTrVal[trainIdx]  #this will work if xTrVal is a numpy array.. if xTrVal is a list, use list comprehension
    xVal = xTrVal[valIdx]
    
    #Use sklearn KNN package to provide training and validation results for range of reasonable k-values

from sklearn.neighbors import KNeighborsClassifier

for i in range(1,200):
    neigh = KNeighborsClassifier(n_neighbors=i)
    neigh.fit(X, y) 
# KNeighborsClassifier(...)
# >>> print(neigh.predict([[1.1]]))
# [0]
# >>> print(neigh.predict_proba([[0.9]]))
# [[ 0.66666667  0.33333333]]
#     #save training and testing results for each of the nFolds runs
    



#Plot all nFolds RMSE (root mean squared error) plots as function of k
#pick k for minimum validation error 
#run xTe, yTe with best k value to confirm error estimate.  

##  2.  Train ElasticNet on breast cancer data set

1.  Use same train test split as above
2.  Use ElasticNetCV to perform cross-validation on xTrVal, yTrVal
3.  Plot the RMS error that ElasticNetCV produces
4.  Extract optimum coefficients and run on test data to confirm performance numbers.  